In [1]:
import requests
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [2]:
# Get data from API

api_dadosabertosurl = 'https://dadosabertos.camara.leg.br/api/v2/deputados'
resposta = requests.request("GET", api_dadosabertosurl, params={})
objetos = json.loads(resposta.text)
dados_df = pd.DataFrame(objetos['dados'])

dados_df = (dados_df[['id', 'nome', 'siglaPartido', "siglaUf"]])

In [12]:
# Create Dataframe
total_despesas_df = pd.DataFrame({
    "id": [],
    "Despesas": []
})

,id,Despesas


In [14]:
# Create new dict
for linha in range(len(dados_df.index)):
    id_deputado = dados_df.loc[linha, 'id']
    api_dadosabertos_despesas = ("https://dadosabertos.camara.leg.br/api/v2/deputados/{}/despesas?ordem=ASC&ordenarPor=ano".format(id_deputado))
    despesas_resposta = requests.get(api_dadosabertos_despesas)
    despesas = despesas_resposta.json()
    despesas_df = pd.DataFrame(despesas['dados'])
    if despesas_df.empty:
        new_row = {'id': id_deputado, 'Despesas': ''}
        total_despesas_df = total_despesas_df._append(new_row, ignore_index=True)
    else:
        total_despesas = despesas_df['valorDocumento'].sum()
        new_row = {'id': id_deputado, 'Despesas': total_despesas}
        total_despesas_df = total_despesas_df._append(new_row, ignore_index=True)
## Need optimization

In [15]:
total_despesas_df

,id,Despesas
0,220593.0,10418.17
1,204379.0,3860.63
2,220714.0,2000.0
3,221328.0,19983.47
4,204560.0,6371.58
...,...,...
508,220558.0,40477.87
509,204517.0,2832.39
510,160592.0,25360.77
511,220592.0,2606.49


In [17]:
dados_df = dados_df.merge(total_despesas_df)
dados_df

,id,nome,siglaPartido,siglaUf,Despesas
0,220593,Abilio Brunini,PL,MT,10418.17
1,204379,Acácio Favacho,MDB,AP,3860.63
2,220714,Adail Filho,REPUBLICANOS,AM,2000.0
3,221328,Adilson Barroso,PL,SP,19983.47
4,204560,Adolfo Viana,PSDB,BA,6371.58
...,...,...,...,...,...
508,220558,Zé Trovão,PL,SC,40477.87
509,204517,Zé Vitor,PL,MG,2832.39
510,160592,Zeca Dirceu,PT,PR,25360.77
511,220592,Zezinho Barbary,PP,AC,2606.49
